# 🔍 RAG 심화 실습: 실시간 웹검색 기반 Retrieval

LangChain의 웹 검색 기능을 활용하여 외부 knowledge source를 동적으로 검색하고 RAG를 구현하는 방법을 다룹니다.

## 🧭 목표
- LangChain의 웹 검색 툴을 활용해 실시간 외부 정보를 Knowledge Source로 사용하는 방법을 익힌다.
- 벡터 DB 기반 RAG과 달리, 정적인 문서 저장 없이도 동적으로 context를 확보하는 RAG 방식 구현

## 1. 라이브러리 설치

In [10]:
!pip uninstall -y langchain-core langchain-openai
!pip install langchain-core==0.3.52 langchain-openai==0.3.12

Found existing installation: langchain-core 0.3.54
Uninstalling langchain-core-0.3.54:
  Successfully uninstalled langchain-core-0.3.54
Found existing installation: langchain-openai 0.3.14
Uninstalling langchain-openai-0.3.14:
  Successfully uninstalled langchain-openai-0.3.14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.4 MB/s eta 0:00:00


## 2. API Key 설정

In [11]:
from dotenv import load_dotenv
import os

load_dotenv()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 3. Tavily Search Tool 초기화

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(api_key=TAVILY_API_KEY)

## 4. 검색 결과를 문서 형식으로 변환

In [13]:
from langchain_core.documents import Document

def search_docs(query: str, k: int = 3):
    results = search_tool.invoke(query)
    return [
        Document(page_content=entry["content"], metadata={"source": entry["url"]})
        for entry in results[:k]
    ]

## 5. Prompt 구성

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Answer the question based on the context below. Cite sources if relevant.

Context:
{context}

Question:
{question}
""")

## 6. RAG Chain 구성

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", api_key=OPENAI_API_KEY)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

web_rag_chain = (
    {
        "context": lambda x: format_docs(search_docs(x["question"])),
        "question": RunnablePassthrough() # 사용자 입력 그대로 전달. LangChain 형식을 맞추기 위함
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

chain = (
    prompt_template
    | llm
    | StrOutputParser()
)

## 7. 테스트 실행

In [23]:
query = {"context": '', "question": " 테슬라 주가 어떻게 될 거 같아?"}
print(chain.invoke(query))

테슬라 주가 전망에 대해서는 여러 요인들이 영향을 미치기 때문에 확실하게 예측하기는 어렵습니다. 일반적으로 테슬라 주가는 전기차 시장 성장 전망, 회사의 실적 발표, 신제품 출시에 따른 기대감, 글로벌 경기 상황, 원자재 가격 변화, 그리고 정책 변화(예: 친환경 관련 보조금 등)에 따라 변동성이 큽니다.

예를 들어, 전기차 수요 증가와 테슬라의 생산 능력 확대, 배터리 기술 혁신 등이 긍정적으로 작용하면 주가가 상승할 가능성이 있습니다. 반대로, 경쟁 심화, 공급망 이슈, 금리 인상과 같은 거시경제 요인들은 주가에 부정적 영향을 줄 수 있습니다.

투자 결정을 하실 때는 최신 뉴스, 애널리스트 리포트, 그리고 자신의 투자 성향과 목표를 고려하는 것이 중요합니다. 또한, 주식 투자는 원금 손실 위험이 있으므로 신중한 판단이 필요합니다.

(참고: [네이버 금융](https://finance.naver.com), [한국거래소](http://www.krx.co.kr))


In [24]:
query = {"question": " 테슬라 주가 어떻게 될 거 같아?"}
print(web_rag_chain.invoke(query))

증권회사들이 테슬라 주가가 더 오를 거라고 보고, 주요 증권사들이 테슬라의 목표주가를 상향 조정했다고 합니다. 따라서 테슬라 주가는 앞으로 더 오를 가능성이 크다고 할 수 있습니다.


## 8. 출처 표시

In [25]:
def format_response_with_sources(answer: str, docs: list) -> str:
    source_list = "\n".join(f"- {doc.metadata['source']}" for doc in docs)
    return f"{answer}\n\n---\n🔗 **출처**\n{source_list}"

# 질문 예시
question = "테슬라 2025년 4월 최근 주가 동향"

# 웹 검색 문서 수집
docs = search_docs(question)

query = {"question": question}
answer = web_rag_chain.invoke(query)

print(format_response_with_sources(answer, docs))


2025년 4월 기준 테슬라의 주가는 최근 큰 폭으로 하락하고 있습니다. 하루 동안 약 4.94% 하락했으며, 2025년 들어 누적 하락률은 약 40%에 달합니다. 다만 2024년 한 해 동안에는 62.5% 상승한 후 일부 조정을 거친 상태입니다. 월가의 평균 목표 주가는 306.62달러, 일부 애널리스트는 최대 360달러까지 제시하고 있어 장기적인 기대감은 여전히 존재합니다.

하락세의 원인으로는 일론 머스크의 정치적 발언과 행동으로 인해 브랜드 이미지가 훼손된 점, 독일, 프랑스, 미국 캘리포니아 등 주요 시장에서의 판매 감소, 중국 BYD 등 현지 브랜드들의 저가 공세, 글로벌 배터리 공급 불안정, 모델 Y 등 생산라인 리툴링 지연 등이 꼽힙니다.

또한, 미국과 중국 간 무역 긴장으로 인한 125% 초고율 관세 부과로 인해 모델 S와 모델 X의 중국 수출 단가가 50% 이상 급등하고, 수요가 최대 75% 급감할 것으로 예상되어 중국 내 고급차 시장 매출 및 수익성에 큰 타격이 발생하고 있습니다.

요약하면, 2025년 4월 들어 테슬라 주가는 단기적으로 큰 폭의 하락세를 보이고 있으나, AI·로보택시 등 신사업에 대한 기대와 월가 목표 주가를 고려할 때 회복 가능성도 주목받고 있는 상황입니다.

출처: 제공된 문맥 내용

---
🔗 **출처**
- https://www.youtube.com/watch?v=b_yvYbod7AM
- https://contents.premium.naver.com/protact/economy/contents/250417101503706fg
- https://www.youtube.com/watch?v=odpsSrNpLic
